<a href="https://colab.research.google.com/github/AlexWortega/Collab_experement/blob/main/Fine_tune_GPT_2_Model_with_Huggingface_pipline%2BBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **What are we going to do:**

- load the dataset from kaggle
- prepare the dataset and build a ``TextDataset``
- load the pre-trained GPT-2 model and tokenizer
- initialize ``Trainer`` with ``TrainingArguments``
- train and save the model
- test the model

In [ ]:
!pip install transformers==4.2.2

     |████████████████████████████████| 1.8 MB 7.5 MB/s 
     |████████████████████████████████| 895 kB 57.4 MB/s 
     |████████████████████████████████| 2.9 MB 28.7 MB/s 


In [ ]:
!nvidia-smi

Mon Aug  9 12:53:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#upload files to your colab environment
from google.colab import files
uploaded = files.upload()

Saving recipes.json to recipes.json


After we uploaded the file with use `unzip` to extract the recipes.json. 

In [ ]:
!unzip '132879_316218_bundle_archive.zip'

unzip:  cannot find or open 132879_316218_bundle_archive.zip, 132879_316218_bundle_archive.zip.zip or 132879_316218_bundle_archive.zip.ZIP.


# Prepare the dataset and build a ``TextDataset``

The next step is to extract the instructions from all recipes and build a `TextDataset`. The `TextDataset` is a custom implementation of the [Pytroch `Dataset` class](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class) implemented by the transformers library. If you want to know more about Dataset in Pytroch you can check out this [youtube video](https://www.youtube.com/watch?v=PXOzkkB5eH0&ab_channel=PythonEngineer).

First, we are going to split the `recipes.json` into a `train` and `test` section and extract `Instructions` from the recipes and write them into a `train_dataset.txt` and `test_dataset.txt`

In [ ]:
import re
import json
from sklearn.model_selection import train_test_split


with open('recipes.json') as f:
    data = json.load(f)

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts['Instructions']).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

train, test = train_test_split(data,test_size=0.15) 


build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))


Train dataset length: 10361
Test dataset length: 1829


the next step is to download the tokenizer, which we use. We use the tokenizer from the `german-gpt2` model on [huggingface](https://huggingface.co/anonymous-german-nlp/german-gpt2).

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

train_path = '/content/train.txt'
test_path = '/content/val (1).txt'

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


# Initialize `Trainer` with `TrainingArguments` and GPT-2 model

The [Trainer](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Trainer) class provides an API for feature-complete training. It is used in most of the [example scripts](https://huggingface.co/transformers/examples.html) from Huggingface. Before we can instantiate our `Trainer` we need to download our GPT-2 model and create a [TrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments) to access all the points of customization during training. In the `TrainingArguments`, we can define the Hyperparameters we are going to use in the training process like our `learning_rate`, `num_train_epochs`, or  `per_device_train_batch_size`. A complete list can you find [here](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments).

In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")


training_args = TrainingArguments(
    output_dir="./gpt2", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


# Train and save the model

To train the model we can simply run `Trainer.train()`.

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=186, training_loss=3.7458263110089045, metrics={'train_runtime': 107.9059, 'train_samples_per_second': 1.724, 'total_flos': 567545676300288, 'epoch': 3.0})

After training is done you can save the model by calling `save_model()`. This will save the trained model to our `output_dir` from our `TrainingArguments`.

In [ ]:
trainer.save_model()

# Test the model

To test the model we are going to use another [highlight of the transformers library](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipelines) called `pipeline`. [Pipelines](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipelines) are objects that offer a simple API dedicated to several tasks, among others also `text-generation`

In [ ]:
import numpy as np
import torch
np.random.seed(42)
torch.manual_seed(42)
from transformers import GPT2LMHeadModel, GPT2Tokenizer

device = 'cuda'

tok = GPT2Tokenizer.from_pretrained("/content/gpt2")
model = GPT2LMHeadModel.from_pretrained("/content/gpt2").to(device)



In [ ]:
!zip -r weights.zip /content/gpt2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def gen(text,len=20):
  text = f'<s>{text}'
  
  temperature = 1.0
  

  inpt = tok.encode(text, return_tensors="pt").to(device)



  out = model.generate(inpt,length = len,max_length = len+30, 
                       
                       do_sample=True,top_k=5000,top_p=0.95,num_beams=3,
                       temperature=temperature,num_return_sequences=3)
 
   
  return tok.decode(out[0])


In [ ]:
!pip install pyTelegramBotAPI
!pip install tg_logger

     |████████████████████████████████| 102 kB 9.1 MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-3.8.2-py3-none-any.whl size=78015 sha256=8fe0008e0aeb8e7978bd4c24cf427c4e7adeb91d2808c7b535a48b75cef03d2a
  Stored in directory: /root/.cache/pip/wheels/05/20/79/0ec340bba3293984eac13cbd480839b754e4edffe1e517f606
Successfully built pyTelegramBotAPI
     |████████████████████████████████| 81 kB 5.8 MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-3.7.6-py3-none-any.whl size=59260 sha256=6257163db57e469b3c50abf8b45c70849275148e4685989303cad734caa18d5c
  Stored in directory: /root/.cache/pip/wheels/65/0a/86/734bdf353913acb2046819291d6e367e3e1b130170b12dd08e
Successfully built pyTelegramBotAPI
  Attempting uninstall: pyTelegramBotAPI
    Found existing installation: pyTelegramBotAPI 3.8.2
    Uninstalling pyTelegramBotAPI-3.8.2:
      Successfully uninstalled pyTelegramBotAPI-3.8.2


# BOT

In [ ]:


from time import sleep

import datetime
import logging
import os
import time
import pytz


import telebot
from telebot import types
import tg_logger
from flask import Flask, request

import re

import random


import numpy as np
import torch
np.random.seed(42)
torch.manual_seed(42)
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tok = GPT2Tokenizer.from_pretrained("/content/gpt2_")
model = GPT2LMHeadModel.from_pretrained("/content/gpt2_").to('cuda')




# ------------- uptime var -------------
boot_time = time.time()
boot_date = datetime.datetime.now(tz=pytz.timezone("Europe/Moscow"))

# ------------- flask config -------------
ADMIN_PASSWORD = 'admin'
app = Flask(__name__)

# ------------- bot config -------------
WEBHOOK_TOKEN = 'aa'
BOT_TOKEN = 'TGTOKEN'
bot = telebot.TeleBot(BOT_TOKEN)

# ------------- log ---------------
users = ['Check docs']

alpha_logger = logging.getLogger()
alpha_logger.setLevel(logging.INFO)
tg_logger.setup(alpha_logger, token="LOGS BOT TOKEN HERE", users=users)
tg_logger.setup(app.logger, token="LOGS BOT TOKEN HERE", users=users)

logger = logging.getLogger("tg-bot-gpt")


# -------------- status webpage --------------
@app.route('/')
def status():
    password = request.args.get("password")
    if password != ADMIN_PASSWORD:
        logger.info('Status page loaded without password')
        return "<h1>Access denied!<h1>", 403

    return f'<h1>This is telegram bot server, ' \
           f'<a href="WTF are u doing here">templated</a> by ' \
           f'<a href="https://github.com/AlexWortega">@alexwortega</a></h1>' \
           f'<p>Server uptime: {datetime.timedelta(seconds=time.time() - boot_time)}</p>' \
           f'<p>Server last boot at {boot_date}'


# ------------- webhook ----------------
@app.route('/' + WEBHOOK_TOKEN, methods=['POST'])
def getMessage():
    temp = request.stream.read().decode("utf-8")
    temp = telebot.types.Update.de_json(temp)
    logger.debug('New message received. raw: %s', temp)
    bot.process_new_updates([temp])
    return "!", 200



def push(text,lenght,temp,num):
  text = f'<s>{text}'
  
  temperature = temp
  

  inpt = tok.encode(text, return_tensors="pt").to('cuda')



  out = model.generate(inpt,length = 1 ,max_length = lenght, 
                       do_sample=True,top_k=5000,top_p=0.95,
                       temperature=temp)
 
   
  return tok.decode(out[0])



@app.route("/set_webhook")
def webhook_on():
    password = request.args.get("password")
    if password != ADMIN_PASSWORD:
        logger.info('Set_webhook page loaded without password')
        return "<h1>Access denied!<h1>", 403

    bot.remove_webhook()
    url = 'https://' + os.environ.get('HOST') + '/' + WEBHOOK_TOKEN
    print(url)
    bot.set_webhook(url=url)
    logger.info(f'Webhook is ON! Url: %s', url)
    return "<h1>WebHook is ON!</h1>", 200


@app.route("/remove_webhook")
def webhook_off():
    password = "admin"
    if password != ADMIN_PASSWORD:
        logger.info('Remove_webhook page loaded without password')
        return "<h1>Access denied!<h1>", 403

    bot.remove_webhook()
    logger.info('WebHook is OFF!')
    return "<h1>WebHook is OFF!</h1>", 200


class Gpt:
  def __init__(self):
    self.len=200
    self.temp=1.1
    self.num=1
  def set_num(self,num:int):
    self.num=num
  def set_temp(self,temp:float):
    self.temp=temp
  def set_len(self,len:int):
    self.len=len


@bot.message_handler(commands=['help', 'start'])
def say_welcome(message):
    
    logger.info(f'</code>@{message.from_user.username}<code> ({message.chat.id}) used /start or /help')
    bot.send_message(message.chat.id,
                     'Добро пожаловать Pushkin replicant bot, пока есть следущие команды:set_temp=[0.0,4.0] , set_lenght=[1,1000] ',
                     parse_mode='html',)

def extract_arg(arg):
    return arg.split()[1:]
gp = Gpt()
@bot.message_handler(commands=['setlen'])
def yourCommand(message):
    status = extract_arg(message.text)
    #print(status[0])
    try:
      gp.set_len(int(status[0]))
      age(message.chat.id,f"len = {gp.len} temp = {gp.temp} num= {gp.num}")
    except:
      bot.send_message(message.chat.id,f'что то пошло не так, попробуй ""/setlen 150""msg = {status} ')
@bot.message_handler(commands=['settemp'])
def yourCommand(message):
    status = extract_arg(message.text)
    #print(status[0])
    try:
      gp.set_temp(float(status[0]))
      age(message.chat.id,f"len = {gp.len} temp = {gp.temp} num= {gp.num}")
    except:
      bot.send_message(message.chat.id,f'что то пошло не так, попробуй ""/settemp 1.4""msg = {status} ')

@bot.message_handler(commands=['setnum'])
def yourCommand(message):
    status = extract_arg(message.text)
    #print(status[0])
    try:
      gp.set_num(int(status[0]))
      bot.send_message(message.chat.id,f"len = {gp.len} temp = {gp.temp} num= {gp.num}")
    except:
      bot.send_message(message.chat.id,f'что то пошло не так, попробуй ""/setnum 1""msg = {status} ')



@bot.message_handler(content_types=['text'])
def process_step(message):
   
    text = message.text
    print(message.text)
    lenght = gp.len
    temp = gp.temp
    
    if lenght>10 and lenght<500 and temp>0.0 and temp<4.0:


      try:  
        reply = push(text=text,lenght=lenght,temp=temp,num=3)
        
        bot.send_message(message.chat.id,reply)
      except:
        gp.set_len(50)
        gp.set_temp(1.2)
        bot.send_message(message.chat.id,'чет сломалось но пара ифов и мой великий ии все починили')


 
    """
    регулярки и стоп слова 
    """
    
    


if __name__ == '__main__':

  
  bot.polling(none_stop=True)


    
        
  

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Как срать


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Как срать


In [ ]:
gen('Соболь и навальный ')